In [ ]:
%pylab inline

# Imports
import pandas as pd
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Populating the interactive namespace from numpy and matplotlib


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
np.random.seed(24)

In [ ]:
root_dir = "/content/gdrive/My Drive/rec_system/"
print("The data lies here =>", root_dir)

The data lies here => /content/gdrive/My Drive/rec_system/


In [ ]:
df_tags = pd.read_csv("/content/gdrive/MyDrive/rec_system/tags.csv", encoding= 'unicode_escape')
df_chapters = pd.read_csv("/content/gdrive/MyDrive/rec_system/chapters.csv", encoding= 'unicode_escape')



In [ ]:
df_chapters.head()

,chapterID,Section,Chapter,Topic,tagID,url
0,cID1,1.1,1,About the Book,1,"""https://www.youtube.com/watch?v=TYG6lxSjO6A&l..."
1,cID2,1.2,1,Systems and Models,2,"""https://www.youtube.com/watch?v=TYG6lxSjO6A&l..."
2,cID3,1.3,1,When to Simulate (and When Not To),3,"""https://www.youtube.com/watch?v=TYG6lxSjO6A&l..."
3,cID4,1.4,1,Simulation Success Skills,4,"""https://www.youtube.com/watch?v=TYG6lxSjO6A&l..."
4,cID5,2.1,2,Queueing - System Structure and Terminology,5,"""https://www.youtube.com/watch?v=1oobmTyKcg8&l..."


In [ ]:
df_chapters["Section"] = df_chapters["Section"]
df_chapters["Chapter"] = df_chapters["Chapter"]
df_chapters["tagID"] = df_chapters["tagID"]

df_chapters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   chapterID  50 non-null     object 
 1   Section    50 non-null     float64
 2   Chapter    50 non-null     int64  
 3   Topic      50 non-null     object 
 4   tagID      50 non-null     int64  
 5   url        50 non-null     object 
dtypes: float64(1), int64(2), object(3)
memory usage: 2.5+ KB


In [ ]:
df_tags.head()

,tagID,tag
0,1,animation
1,2,agility
2,3,arena
3,4,array
4,5,arrivals


In [ ]:
df_tags["tagID"] = df_tags["tagID"].map(str)

In [ ]:
id = []
question_desc = []
for i in range(1, 501, 1):
  id.append(i)
  question_desc.append("This is Question " + str(i))

print(id)
print(question_desc)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

In [ ]:
exam_1 = []
exam_2 = []
final = []

for i in range (500):
  exam_1.append(random.randint(0,1))
  exam_2.append(random.randint(0,1))
  final.append(random.randint(0,1))

print(exam_1)
print(exam_2)
print(final)

[1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 

In [ ]:
df_chapters["Section"] = df_chapters["Section"].map(str)
df_chapters["Chapter"] = df_chapters["Chapter"].map(str)
df_chapters["tagID"] = df_chapters["tagID"].map(str)

df_chapters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   chapterID  50 non-null     object
 1   Section    50 non-null     object
 2   Chapter    50 non-null     object
 3   Topic      50 non-null     object
 4   tagID      50 non-null     object
 5   url        50 non-null     object
dtypes: object(6)
memory usage: 2.5+ KB


In [ ]:
topic_id = df_chapters['chapterID'].tolist()

print(topic_id)

['cID1', 'cID2', 'cID3', 'cID4', 'cID5', 'cID6', 'cID7', 'cID8', 'cID9', 'cID10', 'cID11', 'cID12', 'cID13', 'cID14', 'cID15', 'cID16', 'cID17', 'cID18', 'cID19', 'cID20', 'cID21', 'cID22', 'cID23', 'cID24', 'cID25', 'cID26', 'cID27', 'cID28', 'cID29', 'cID30', 'cID31', 'cID32', 'cID33', 'cID34', 'cID35', 'cID36', 'cID37', 'cID38', 'cID39', 'cID40', 'cID41', 'cID42', 'cID43', 'cID44', 'cID45', 'cID46', 'cID47', 'cID48', 'cID49', 'cID50']


In [ ]:
topic_1 = []
topic_2 = []
points = []
median = []
num_scores = []

for i in range(500):
  topic_1.extend(random.choices(topic_id))
  topic_2.extend(random.choices(topic_id))
  points.append(random.randint(0,5))
  median.append(random.randint(2,5))
  num_scores.append(random.randint(27,99))

print(topic_1)
print(topic_2)
print(points)

['cID23', 'cID42', 'cID12', 'cID36', 'cID45', 'cID18', 'cID16', 'cID35', 'cID9', 'cID26', 'cID47', 'cID37', 'cID15', 'cID42', 'cID12', 'cID43', 'cID6', 'cID34', 'cID40', 'cID21', 'cID41', 'cID33', 'cID21', 'cID5', 'cID3', 'cID28', 'cID1', 'cID23', 'cID12', 'cID13', 'cID22', 'cID48', 'cID11', 'cID8', 'cID39', 'cID43', 'cID38', 'cID28', 'cID50', 'cID38', 'cID47', 'cID20', 'cID16', 'cID16', 'cID10', 'cID36', 'cID21', 'cID19', 'cID17', 'cID31', 'cID25', 'cID10', 'cID45', 'cID49', 'cID11', 'cID9', 'cID45', 'cID7', 'cID24', 'cID16', 'cID15', 'cID19', 'cID37', 'cID31', 'cID7', 'cID6', 'cID37', 'cID22', 'cID42', 'cID9', 'cID4', 'cID41', 'cID5', 'cID19', 'cID36', 'cID13', 'cID50', 'cID7', 'cID1', 'cID25', 'cID37', 'cID21', 'cID48', 'cID3', 'cID1', 'cID7', 'cID16', 'cID12', 'cID13', 'cID23', 'cID10', 'cID9', 'cID5', 'cID5', 'cID9', 'cID49', 'cID36', 'cID19', 'cID27', 'cID28', 'cID16', 'cID35', 'cID10', 'cID49', 'cID37', 'cID37', 'cID13', 'cID46', 'cID25', 'cID28', 'cID32', 'cID27', 'cID29', 'cID

In [ ]:
df_questions = pd.DataFrame(list(zip(id, question_desc, topic_1, topic_2, points, exam_1, exam_2, final)),
               columns =['ID', 'Desc', 'Topic1', 'Topic2', "Average Score", "Exam 1", "Exam 2", "Final"])

In [ ]:
df_questions.head()

,ID,Desc,Topic1,Topic2,Average Score,Exam 1,Exam 2,Final
0,1,This is Question 1,cID23,cID5,0,1,0,0
1,2,This is Question 2,cID42,cID50,2,0,1,0
2,3,This is Question 3,cID12,cID39,0,0,0,1
3,4,This is Question 4,cID36,cID32,2,1,0,1
4,5,This is Question 5,cID45,cID21,0,1,0,1


In [ ]:
df_with_top = pd.merge(df_questions, df_chapters, left_on="Topic1", right_on="chapterID", how='left', copy = True)

In [ ]:
df_with_top.head()

,ID,Desc,Topic1,Topic2,Average Score,Exam 1,Exam 2,Final,chapterID,Section,Chapter,Topic,tagID,url
0,1,This is Question 1,cID23,cID5,0,1,0,0,cID23,5.2,5,First Project Using the Standard Library Objects,23,"""https://www.youtube.com/watch?v=x8R32Z-o9Lo&l..."
1,2,This is Question 2,cID42,cID50,2,0,1,0,cID42,7.5,7,Lists,42,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l..."
2,3,This is Question 3,cID12,cID39,0,0,0,1,cID12,3.2,3,Manual Simulation,12,"""https://www.youtube.com/watch?v=qbfHVvJ2Ez4&l..."
3,4,This is Question 4,cID36,cID32,2,1,0,1,cID36,6.6,6,Summary,36,"""https://www.youtube.com/watch?v=JLtEFlTN9Y4&l..."
4,5,This is Question 5,cID45,cID21,0,1,0,1,cID45,7.8,7,Summary,45,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l..."


In [ ]:
df_with_top = pd.merge(df_with_top, df_chapters, left_on="Topic2", right_on="chapterID", how='left', copy = True, suffixes=("_1", "_2"))

In [ ]:
df_with_top.head()

,ID,Desc,Topic1,Topic2,Average Score,Exam 1,Exam 2,Final,chapterID_1,Section_1,Chapter_1,Topic_1,tagID_1,url_1,chapterID_2,Section_2,Chapter_2,Topic_2,tagID_2,url_2
0,1,This is Question 1,cID23,cID5,0,1,0,0,cID23,5.2,5,First Project Using the Standard Library Objects,23,"""https://www.youtube.com/watch?v=x8R32Z-o9Lo&l...",cID5,2.1,2,Queueing - System Structure and Terminology,5,"""https://www.youtube.com/watch?v=1oobmTyKcg8&l..."
1,2,This is Question 2,cID42,cID50,2,0,1,0,cID42,7.5,7,Lists,42,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l...",cID50,8.4,8,ED Model Revisited,50,"""https://www.youtube.com/watch?v=t3hTCwYGeF8&l..."
2,3,This is Question 3,cID12,cID39,0,0,0,1,cID12,3.2,3,Manual Simulation,12,"""https://www.youtube.com/watch?v=qbfHVvJ2Ez4&l...",cID39,7.2,7,Schedules,39,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l..."
3,4,This is Question 4,cID36,cID32,2,1,0,1,cID36,6.6,6,Summary,36,"""https://www.youtube.com/watch?v=JLtEFlTN9Y4&l...",cID32,6.2,6,PCB Assembly,32,"""https://www.youtube.com/watch?v=JLtEFlTN9Y4&l..."
4,5,This is Question 5,cID45,cID21,0,1,0,1,cID45,7.8,7,Summary,45,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l...",cID21,4.5,4,Problems,21,"""https://www.youtube.com/watch?v=6DN1NYfG1kQ&l..."


In [ ]:
df_with_top = pd.merge(df_with_top, df_tags, left_on="tagID_1", right_on="tagID", how='left', copy = True, suffixes=("_x", "_y"))

In [ ]:
df_with_top = pd.merge(df_with_top, df_tags, left_on="tagID_2", right_on="tagID", how='left', copy = True, suffixes=("_x", "_y"))

In [ ]:
df_with_top.head()

,ID,Desc,Topic1,Topic2,Average Score,Exam 1,Exam 2,Final,chapterID_1,Section_1,...,chapterID_2,Section_2,Chapter_2,Topic_2,tagID_2,url_2,tagID_x,tag_x,tagID_y,tag_y
0,1,This is Question 1,cID23,cID5,0,1,0,0,cID23,5.2,...,cID5,2.1,2,Queueing - System Structure and Terminology,5,"""https://www.youtube.com/watch?v=1oobmTyKcg8&l...",23,excel,5,arrivals
1,2,This is Question 2,cID42,cID50,2,0,1,0,cID42,7.5,...,cID50,8.4,8,ED Model Revisited,50,"""https://www.youtube.com/watch?v=t3hTCwYGeF8&l...",42,poisson process,50,risk analysis
2,3,This is Question 3,cID12,cID39,0,0,0,1,cID12,3.2,...,cID39,7.2,7,Schedules,39,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l...",12,correlation,39,linear
3,4,This is Question 4,cID36,cID32,2,1,0,1,cID36,6.6,...,cID32,6.2,6,PCB Assembly,32,"""https://www.youtube.com/watch?v=JLtEFlTN9Y4&l...",36,oculus,32,integration
4,5,This is Question 5,cID45,cID21,0,1,0,1,cID45,7.8,...,cID21,4.5,4,Problems,21,"""https://www.youtube.com/watch?v=6DN1NYfG1kQ&l...",45,queue,21,state statistics


In [ ]:
df_with_top["Topics"] = df_with_top["Topic_1"] + ", " + df_with_top["Topic_2"]
# df_with_top["chapterIDs"] = df_with_top["chapterID_1"] + ", " + df_with_top["chapterID_2"]
df_with_top["Chapters"] = df_with_top["Chapter_1"] + ", " + df_with_top["Chapter_2"]
df_with_top["Sections"] = df_with_top["Section_1"] # + ", " + df_with_top["Section_2"]
df_with_top["URLs"] = df_with_top["url_1"] + ", " + df_with_top["url_2"]
df_with_top["tags"] = df_with_top["tag_x"] + ", " + df_with_top["tag_y"]
df_with_top["tagIDs"] = df_with_top["tagID_1"] + " " + df_with_top["tagID_2"]

df_with_top = df_with_top.drop(["Topic1", "Topic2", "Topic_1", "Topic_2", "chapterID_1", "chapterID_2", "Chapter_1", "Chapter_2", "Section_1", "Section_2", "url_1", "url_2", "tag_x", "tag_y", "tagID_1", "tagID_2", "tagID_x", "tagID_y"], axis=1)
df_with_top.head()

,ID,Desc,Average Score,Exam 1,Exam 2,Final,Topics,Chapters,Sections,URLs,tags,tagIDs
0,1,This is Question 1,0,1,0,0,First Project Using the Standard Library Objec...,"5, 2",5.2,"""https://www.youtube.com/watch?v=x8R32Z-o9Lo&l...","excel, arrivals",23 5
1,2,This is Question 2,2,0,1,0,"Lists, ED Model Revisited","7, 8",7.5,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l...","poisson process, risk analysis",42 50
2,3,This is Question 3,0,0,0,1,"Manual Simulation, Schedules","3, 7",3.2,"""https://www.youtube.com/watch?v=qbfHVvJ2Ez4&l...","correlation, linear",12 39
3,4,This is Question 4,2,1,0,1,"Summary, PCB Assembly","6, 6",6.6,"""https://www.youtube.com/watch?v=JLtEFlTN9Y4&l...","oculus, integration",36 32
4,5,This is Question 5,0,1,0,1,"Summary, Problems","7, 4",7.8,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l...","queue, state statistics",45 21


In [ ]:
df_with_top.to_csv("/content/gdrive/MyDrive/rec_system/questions.csv", index=False)

In [ ]:
df_with_top = df_with_top.drop(["tagIDs", "Chapters"], axis=1)
df_with_top.head()

,ID,Desc,Average Score,Exam 1,Exam 2,Final,Topics,Sections,URLs,tags
0,1,This is Question 1,0,1,0,0,First Project Using the Standard Library Objec...,5.2,"""https://www.youtube.com/watch?v=x8R32Z-o9Lo&l...","excel, arrivals"
1,2,This is Question 2,2,0,1,0,"Lists, ED Model Revisited",7.5,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l...","poisson process, risk analysis"
2,3,This is Question 3,0,0,0,1,"Manual Simulation, Schedules",3.2,"""https://www.youtube.com/watch?v=qbfHVvJ2Ez4&l...","correlation, linear"
3,4,This is Question 4,2,1,0,1,"Summary, PCB Assembly",6.6,"""https://www.youtube.com/watch?v=JLtEFlTN9Y4&l...","oculus, integration"
4,5,This is Question 5,0,1,0,1,"Summary, Problems",7.8,"""https://www.youtube.com/watch?v=Y46g5Np_euY&l...","queue, state statistics"


In [ ]:
df_with_top.to_csv("/content/gdrive/MyDrive/rec_system/questions2.csv", index=False)